In [32]:
!pip install openpyxl

     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     ------------ -------------------------- 81.9/250.0 kB 2.3 MB/s eta 0:00:01
     ------------ -------------------------- 81.9/250.0 kB 2.3 MB/s eta 0:00:01
     ---------------- ------------------- 112.6/250.0 kB 726.2 kB/s eta 0:00:01
     ---------------- ------------------- 112.6/250.0 kB 726.2 kB/s eta 0:00:01
     -------------------- --------------- 143.4/250.0 kB 607.9 kB/s eta 0:00:01
     ----------------------- ------------ 163.8/250.0 kB 656.4 kB/s eta 0:00:01
     ----------------------------- ------ 204.8/250.0 kB 593.2 kB/s eta 0:00:01
     -------------------------------- --- 225.3/250.0 kB 655.6 kB/s eta 0:00:01
     ------------------------------------ 250.0/250.0 kB 613.2 kB/s eta 0:00:00


In [44]:
import re 
import pandas as pd
from datetime import datetime

In [45]:
file = open('log1.txt', "r")
lines = file.readlines()

In [46]:
l = []
for i in lines:
    if (i.find("Query") != -1 or i.find("recycle") != -1) and i.find("took") == -1:
        l.append(i)  

In [50]:
timestamps, query_num, recycles, pLDDTs, pTMs, ipTMs, tols, length = [], [], [], [], [], [], [], []

for i in l:
    if i.find("Query") != -1:
        query_match = re.search(r'Query (\d+)', i)
        if query_match:
            query_number = query_match.group(1)
        length_match = re.search(r'length (\d+)', i)
        if length_match:
            length_value = length_match.group(1)
    else:
        match = re.search(r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}).*recycle=(\d+) pLDDT=([\d.]+) pTM=([\d.]+) ipTM=([\d.]+)( tol=([\d.]+))?", i)
        if match:
            # print(match.group(1))
            # print(match.group(2))
            # print(match.group(3))
            # print(match.group(4))
            # print(match.group(5))
            # print(match.group(6))
            timestamps.append(match.group(1))
            recycles.append(float(match.group(2)))
            pLDDTs.append(float(match.group(3)))
            pTMs.append(float(match.group(4)))
            ipTMs.append(float(match.group(5)))
            tols.append(float(match.group(7)) if match.group(7) else None)
            query_num.append(float(query_number))
            length.append(float(length_value))
# Create a DataFrame
df = pd.DataFrame({
    'Timestamp': timestamps,
    'Query_num': query_num,
    'Length' : length,
    'Recycle': recycles,
    'pLDDT': pLDDTs,
    'pTM': pTMs,
    'ipTM': ipTMs,
    'Tol': tols
})
def convert_to_ts(str):
    return datetime.strptime(str, "%Y-%m-%d %H:%M:%S,%f")
df['Timestamp_final'] = df['Timestamp'].apply(convert_to_ts)
df.drop(columns=["Timestamp"] , inplace = True)
df.to_excel("Export.xlsx", index = False)